In [1]:
!pip install unrar

In [3]:
!unrar x /content/drive/MyDrive/dataset_B_Eye_Images.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/dataset_B_Eye_Images.rar

Creating    dataset_B_Eye_Images                                      OK
Creating    dataset_B_Eye_Images/closedLeftEyes                       OK
Extracting  dataset_B_Eye_Images/closedLeftEyes/closed_eye_0001.jpg_face_1_L.jpg       0%  OK 
Extracting  dataset_B_Eye_Images/closedLeftEyes/closed_eye_0002.jpg_face_2_L.jpg       0%  OK 
Extracting  dataset_B_Eye_Images/closedLeftEyes/closed_eye_0003.jpg_face_2_L.jpg       0%  OK 
Extracting  dataset_B_Eye_Images/closedLeftEyes/closed_eye_0007.jpg_face_1_L.jpg       0%  OK 
Extracting  dataset_B_Eye_Images/closedLeftEyes/closed_eye_0009.jpg_face_1_L.jpg       0%  OK 
Extracting  dataset_B_Eye_Images/closedLeftEyes/closed_eye_0012.jpg_face_1_L.jpg       0%  OK 
Extracting  dataset_B_Eye_Images/closedLeftEyes/closed_eye_0013.BMP_face_1_L.jpg     

In [4]:
import os
import shutil

# --- CẤU HÌNH ĐƯỜNG DẪN ---
# Thư mục gốc nơi bạn vừa giải nén file rar
# Nếu bạn dùng lệnh giải nén của tôi ở bước trước thì nó là 'CEW_extracted/dataset_B'
# (Lưu ý: CEW thường có folder con là 'dataset_B' sau khi giải nén)
SOURCE_ROOT = '/content/dataset_B_Eye_Images'

# Nơi bạn muốn lưu dataset đã gộp gọn gàng
DEST_ROOT = 'CEW_clean'

# Định nghĩa các folder nguồn và đích
# Cấu trúc: 'Tên folder gốc trong CEW': 'Tên folder đích mong muốn'
merge_map = {
    'closedLeftEyes': 'closed',
    'closedRightEyes': 'closed',
    'openLeftEyes': 'open',
    'openRightEyes': 'open'
}

# --- BẮT ĐẦU XỬ LÝ ---
# 1. Tạo thư mục đích
if not os.path.exists(DEST_ROOT):
    os.makedirs(DEST_ROOT)

for src_name, dest_name in merge_map.items():
    src_path = os.path.join(SOURCE_ROOT, src_name)
    dest_path = os.path.join(DEST_ROOT, dest_name)

    # Tạo folder con (open/closed) nếu chưa có
    os.makedirs(dest_path, exist_ok=True)

    # Kiểm tra xem folder nguồn có tồn tại không
    if not os.path.exists(src_path):
        print(f"⚠️ Cảnh báo: Không tìm thấy thư mục {src_path}. Hãy kiểm tra lại đường dẫn giải nén.")
        continue

    print(f"🔄 Đang chuyển ảnh từ [{src_name}] sang [{dest_name}]...")

    # Duyệt và di chuyển file
    files = os.listdir(src_path)
    count = 0
    for filename in files:
        # Bỏ qua các file hệ thống ẩn nếu có
        if filename.startswith('.'): continue

        old_file = os.path.join(src_path, filename)

        # Đổi tên file để tránh trùng lặp (ví dụ: left_001.jpg)
        new_filename = f"{src_name}_{filename}"
        new_file = os.path.join(dest_path, new_filename)

        shutil.copy2(old_file, new_file) # Dùng copy2 để giữ nguyên metadata, dùng move nếu muốn cắt
        count += 1

    print(f"   ✅ Đã chép xong {count} ảnh.")

print("\n" + "="*30)
print("🎉 HOÀN TẤT GỘP DỮ LIỆU!")
print(f"Dữ liệu mới nằm tại thư mục: {DEST_ROOT}")

# In thống kê nhanh
for category in ['closed', 'open']:
    path = os.path.join(DEST_ROOT, category)
    if os.path.exists(path):
        print(f"- Folder '{category}': {len(os.listdir(path))} ảnh")

🔄 Đang chuyển ảnh từ [closedLeftEyes] sang [closed]...
   ✅ Đã chép xong 1193 ảnh.
🔄 Đang chuyển ảnh từ [closedRightEyes] sang [closed]...
   ✅ Đã chép xong 1193 ảnh.
🔄 Đang chuyển ảnh từ [openLeftEyes] sang [open]...
   ✅ Đã chép xong 1232 ảnh.
🔄 Đang chuyển ảnh từ [openRightEyes] sang [open]...
   ✅ Đã chép xong 1232 ảnh.

🎉 HOÀN TẤT GỘP DỮ LIỆU!
Dữ liệu mới nằm tại thư mục: CEW_clean
- Folder 'closed': 2386 ảnh
- Folder 'open': 2464 ảnh


In [2]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.7 MB/s eta 0:00:00


In [12]:
import os
import cv2
import numpy as np
import onnxruntime as ort
import time  # <--- THƯ VIỆN ĐỂ ĐO GIỜ

# --- CẤU HÌNH ---
MODEL_PATH = 'ocec_s.onnx'
DATA_DIR = 'CEW_clean'
INPUT_H, INPUT_W = 24, 40

# Load Model
print(f"⏳ Đang load model từ {MODEL_PATH}...")
session = ort.InferenceSession(MODEL_PATH, providers=['CPUExecutionProvider'])
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
print("✅ Load model thành công!")

# --- HÀM XỬ LÝ ---
def preprocess(image_path):
    img = cv2.imread(image_path)
    if img is None: return None
    img = cv2.resize(img, (INPUT_W, INPUT_H))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = img.transpose(2, 0, 1)
    img = np.expand_dims(img, axis=0)
    return img

def predict_raw(img_tensor):
    # Hàm này chỉ chạy inference để đo thời gian cho chính xác
    outputs = session.run([output_name], {input_name: img_tensor})
    return outputs[0][0]

# --- BƯỚC KHỞI ĐỘNG (WARM-UP) ---
# Model ONNX thường chạy rất chậm ở lần đầu tiên (do khởi tạo graph),
# nên ta cần chạy nháp 1 lần để số liệu đo sau này chính xác.
print("🔥 Đang khởi động model (Warm-up)...")
dummy_input = np.random.randn(1, 3, 24, 40).astype(np.float32)
session.run([output_name], {input_name: dummy_input})
print("✅ Đã khởi động xong. Bắt đầu benchmark...")

# --- CHẠY TEST ---
classes = {'closed': 0, 'open': 1}
results = {'correct': 0, 'total': 0}

# Danh sách lưu thời gian suy luận của từng ảnh
inference_times = []

# Bắt đầu bấm giờ toàn bộ dataset (tính cả thời gian đọc file, xử lý...)
total_dataset_start = time.time()

for label_str, label_idx in classes.items():
    folder = os.path.join(DATA_DIR, label_str)
    if not os.path.exists(folder): continue

    print(f"--> Đang xử lý class: {label_str}...")
    files = os.listdir(folder)

    for fname in files:
        fpath = os.path.join(folder, fname)

        # 1. Tiền xử lý (Không tính vào thời gian inference của model)
        img_input = preprocess(fpath)
        if img_input is None: continue

        # 2. Đo thời gian suy luận (Inference Time)
        t_start = time.perf_counter() # Bấm giờ

        raw_output = predict_raw(img_input)

        t_end = time.perf_counter()   # Dừng giờ

        # Lưu thời gian (đổi ra mili-giây)
        inference_times.append((t_end - t_start) * 1000)

        # Xử lý kết quả
        if isinstance(raw_output, (np.ndarray, list)) and len(raw_output) > 1:
            pred_idx = np.argmax(raw_output)
        else:
            val = raw_output if np.isscalar(raw_output) else raw_output[0]
            pred_idx = 0 if val < 0.5 else 1 # Giả sử sigmoid output

        if pred_idx == label_idx:
            results['correct'] += 1
        results['total'] += 1

# Dừng bấm giờ toàn bộ
total_dataset_end = time.time()
total_wall_time = total_dataset_end - total_dataset_start

# --- TÍNH TOÁN THỐNG KÊ ---
if results['total'] > 0:
    acc = (results['correct'] / results['total']) * 100

    # Tính toán thời gian trung bình
    avg_inf_time = sum(inference_times) / len(inference_times)
    min_inf_time = min(inference_times)
    max_inf_time = max(inference_times)

    # Tính FPS (Số ảnh xử lý được trong 1 giây dựa trên tốc độ model)
    fps = 1000 / avg_inf_time

    print("\n" + "="*40)
    print(f"📊 KẾT QUẢ HIỆU NĂNG (BENCHMARK)")
    print("="*40)
    print(f"1. Độ chính xác (Accuracy):  {acc:.2f}% ({results['correct']}/{results['total']})")
    print("-" * 40)
    print(f"2. Thời gian suy luận (chỉ tính model chạy):")
    print(f"   - Trung bình 1 ảnh:       {avg_inf_time:.4f} ms (mili-giây)")
    print(f"   - Nhanh nhất:             {min_inf_time:.4f} ms")
    print(f"   - Chậm nhất:              {max_inf_time:.4f} ms")
    print(f"   - Tốc độ lý thuyết (FPS): {fps:.2f} frames/sec")
    print("-" * 40)
    print(f"3. Tổng thời gian chạy dataset (bao gồm cả đọc ảnh):")
    print(f"   - Tổng cộng:              {total_wall_time:.2f} giây")
    print(f"   - Số ảnh đã xử lý:        {results['total']} ảnh")
    print("="*40)
else:
    print("Không có dữ liệu để đo.")

⏳ Đang load model từ ocec_s.onnx...
✅ Load model thành công!
🔥 Đang khởi động model (Warm-up)...
✅ Đã khởi động xong. Bắt đầu benchmark...
--> Đang xử lý class: closed...
--> Đang xử lý class: open...

📊 KẾT QUẢ HIỆU NĂNG (BENCHMARK)
1. Độ chính xác (Accuracy):  71.79% (3479/4846)
----------------------------------------
2. Thời gian suy luận (chỉ tính model chạy):
   - Trung bình 1 ảnh:       1.3852 ms (mili-giây)
   - Nhanh nhất:             0.9798 ms
   - Chậm nhất:              11.6984 ms
   - Tốc độ lý thuyết (FPS): 721.91 frames/sec
----------------------------------------
3. Tổng thời gian chạy dataset (bao gồm cả đọc ảnh):
   - Tổng cộng:              7.69 giây
   - Số ảnh đã xử lý:        4846 ảnh


In [ ]:
import os
import cv2
import numpy as np
import onnxruntime as ort
import time  # <--- THƯ VIỆN ĐỂ ĐO GIỜ

# --- CẤU HÌNH ---
MODEL_PATH = 'ocec_l.onnx'
DATA_DIR = 'CEW_clean'
INPUT_H, INPUT_W = 24, 40

# Load Model
print(f"⏳ Đang load model từ {MODEL_PATH}...")
session = ort.InferenceSession(MODEL_PATH, providers=['CPUExecutionProvider'])
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
print("✅ Load model thành công!")

# --- HÀM XỬ LÝ ---
def preprocess(image_path):
    img = cv2.imread(image_path)
    if img is None: return None
    img = cv2.resize(img, (INPUT_W, INPUT_H))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = img.transpose(2, 0, 1)
    img = np.expand_dims(img, axis=0)
    return img

def predict_raw(img_tensor):
    # Hàm này chỉ chạy inference để đo thời gian cho chính xác
    outputs = session.run([output_name], {input_name: img_tensor})
    return outputs[0][0]

# --- BƯỚC KHỞI ĐỘNG (WARM-UP) ---
# Model ONNX thường chạy rất chậm ở lần đầu tiên (do khởi tạo graph),
# nên ta cần chạy nháp 1 lần để số liệu đo sau này chính xác.
print("🔥 Đang khởi động model (Warm-up)...")
dummy_input = np.random.randn(1, 3, 24, 40).astype(np.float32)
session.run([output_name], {input_name: dummy_input})
print("✅ Đã khởi động xong. Bắt đầu benchmark...")

# --- CHẠY TEST ---
classes = {'closed': 0, 'open': 1}
results = {'correct': 0, 'total': 0}

# Danh sách lưu thời gian suy luận của từng ảnh
inference_times = []

# Bắt đầu bấm giờ toàn bộ dataset (tính cả thời gian đọc file, xử lý...)
total_dataset_start = time.time()

for label_str, label_idx in classes.items():
    folder = os.path.join(DATA_DIR, label_str)
    if not os.path.exists(folder): continue

    print(f"--> Đang xử lý class: {label_str}...")
    files = os.listdir(folder)

    for fname in files:
        fpath = os.path.join(folder, fname)

        # 1. Tiền xử lý (Không tính vào thời gian inference của model)
        img_input = preprocess(fpath)
        if img_input is None: continue

        # 2. Đo thời gian suy luận (Inference Time)
        t_start = time.perf_counter() # Bấm giờ

        raw_output = predict_raw(img_input)

        t_end = time.perf_counter()   # Dừng giờ

        # Lưu thời gian (đổi ra mili-giây)
        inference_times.append((t_end - t_start) * 1000)

        # Xử lý kết quả
        if isinstance(raw_output, (np.ndarray, list)) and len(raw_output) > 1:
            pred_idx = np.argmax(raw_output)
        else:
            val = raw_output if np.isscalar(raw_output) else raw_output[0]
            pred_idx = 0 if val < 0.5 else 1 # Giả sử sigmoid output

        if pred_idx == label_idx:
            results['correct'] += 1
        results['total'] += 1

# Dừng bấm giờ toàn bộ
total_dataset_end = time.time()
total_wall_time = total_dataset_end - total_dataset_start

# --- TÍNH TOÁN THỐNG KÊ ---
if results['total'] > 0:
    acc = (results['correct'] / results['total']) * 100

    # Tính toán thời gian trung bình
    avg_inf_time = sum(inference_times) / len(inference_times)
    min_inf_time = min(inference_times)
    max_inf_time = max(inference_times)

    # Tính FPS (Số ảnh xử lý được trong 1 giây dựa trên tốc độ model)
    fps = 1000 / avg_inf_time

    print("\n" + "="*40)
    print(f"📊 KẾT QUẢ HIỆU NĂNG (BENCHMARK)")
    print("="*40)
    print(f"1. Độ chính xác (Accuracy):  {acc:.2f}% ({results['correct']}/{results['total']})")
    print("-" * 40)
    print(f"2. Thời gian suy luận (chỉ tính model chạy):")
    print(f"   - Trung bình 1 ảnh:       {avg_inf_time:.4f} ms (mili-giây)")
    print(f"   - Nhanh nhất:             {min_inf_time:.4f} ms")
    print(f"   - Chậm nhất:              {max_inf_time:.4f} ms")
    print(f"   - Tốc độ lý thuyết (FPS): {fps:.2f} frames/sec")
    print("-" * 40)
    print(f"3. Tổng thời gian chạy dataset (bao gồm cả đọc ảnh):")
    print(f"   - Tổng cộng:              {total_wall_time:.2f} giây")
    print(f"   - Số ảnh đã xử lý:        {results['total']} ảnh")
    print("="*40)
else:
    print("Không có dữ liệu để đo.")

⏳ Đang load model từ ocec_l.onnx...
✅ Load model thành công!
🔥 Đang khởi động model (Warm-up)...
✅ Đã khởi động xong. Bắt đầu benchmark...
--> Đang xử lý class: closed...
--> Đang xử lý class: open...

📊 KẾT QUẢ HIỆU NĂNG (BENCHMARK)
1. Độ chính xác (Accuracy):  73.34% (3554/4846)
----------------------------------------
2. Thời gian suy luận (chỉ tính model chạy):
   - Trung bình 1 ảnh:       2.7558 ms (mili-giây)
   - Nhanh nhất:             1.9608 ms
   - Chậm nhất:              21.9888 ms
   - Tốc độ lý thuyết (FPS): 362.87 frames/sec
----------------------------------------
3. Tổng thời gian chạy dataset (bao gồm cả đọc ảnh):
   - Tổng cộng:              14.94 giây
   - Số ảnh đã xử lý:        4846 ảnh


In [13]:
fpath = "/content/drive/MyDrive/DoAn/test_image/opened_eye_face.jpg"

        # 1. Tiền xử lý (Không tính vào thời gian inference của model)
t_start = time.perf_counter()
img_input = preprocess(fpath)
# 2. Đo thời gian suy luận (Inference Time)
 # Bấm giờ

raw_output = predict_raw(img_input)

t_end = time.perf_counter()   # Dừng giờ

# Lưu thời gian (đổi ra mili-giây)
print((t_end - t_start) * 1000)

# Xử lý kết quả
if isinstance(raw_output, (np.ndarray, list)) and len(raw_output) > 1:
    pred_idx = np.argmax(raw_output)
else:
    val = raw_output if np.isscalar(raw_output) else raw_output[0]
    pred_idx = 0 if val < 0.5 else 1 # Giả sử sigmoid output

if pred_idx == 0:
  print("Closed")
else:
  print("Open")

65.5536790000042
Closed
